In [1]:
from riptide import *

In [2]:
iCdG692 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdG692.json')
for rxn in iCdG692.boundary: rxn.bounds = (-1000., 1000.)

### Base Model Statistics

In [3]:
iCdG692

Name,iCdG790
Memory address,0x07f53371a8c90
Number of metabolites,1132
Number of reactions,1129
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"cytosol, extracellular"


In [8]:
# Report some additional stats
exch = 0
for rxn in iCdG692.reactions:
    if len(list(rxn.products)) == 0:
        exch += 1
trans = 0
for rxn in iCdG692.reactions:
    comps = set([x.compartment for x in list(rxn.reactants)] + [x.compartment for x in list(rxn.products)])
    if len(comps) > 1:
        trans += 1
metab = len(list(iCdG692.reactions)) - exch - trans

print('Genes: ' + str(len(list(iCdG692.genes))))
print('Exchanges: ' + str(exch))
print('Transporters: ' + str(trans))
print('Metabolic reactions: ' + str(metab))

Genes: 692
Exchanges: 162
Transporters: 174
Metabolic reactions: 972


In [4]:
x = 0
for cpd in iCdG692.metabolites:
    x += len(cpd.reactions)
print(float(x)/float(len(iCdG692.metabolites))) 

4.5959514170040485


In [7]:
base_obj_val = str(round(iCdG692.slim_optimize(),3))
print('Objective value: ' + base_obj_val)

Objective value: 194.795


In [9]:
doubling = round((1.0 / iCdG692.slim_optimize()) * 3200.0, 2)
print('Doubling time: ' + str(doubling) + ' minutes')

Doubling time: 16.43 minutes


### RIPTiDe *in vivo* Contextualization

In [11]:
# Read in in vivo C. difficile transcription
clindamycin = riptide.read_transcription_file('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/transcript/jenior_etal/clindamycin.format.tsv', 
                                              header=True)
streptomycin = riptide.read_transcription_file('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/transcript/jenior_etal/streptomycin.format.tsv', 
                                               header=True)
gnotobiotic = riptide.read_transcription_file('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/transcript/jenior_etal/gnotobiotic.format.tsv', 
                                               header=True)

In [12]:
clinda_iCdG692_riptide = riptide.contextualize(model=iCdG692, transcriptome=clindamycin)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 316 from 1308 (75.84% change)
Metabolites pruned to 318 from 1235 (74.25% change)
Flux through the objective DECREASED to ~159.4 from ~194.8 (18.17% change)
Context-specific metabolism correlates with transcriptome (r=0.178, p=0.001 *)

RIPTiDe completed in 23 seconds



In [13]:
doubling = round((1.0 / clinda_iCdG692_riptide.model.slim_optimize()) * 3200.0, 2)
print('Doubling time: ' + str(doubling) + ' minutes')

Doubling time: 20.08 minutes


In [14]:
strep_iCdG692_riptide = riptide.contextualize(model=iCdG692, transcriptome=streptomycin)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 313 from 1308 (76.07% change)
Metabolites pruned to 317 from 1235 (74.33% change)
Flux through the objective DECREASED to ~158.28 from ~194.8 (18.75% change)
Context-specific metabolism does not correlate with transcriptome (r=0.094, n.s.)

RIPTiDe completed in 27 seconds



In [15]:
doubling = round((1.0 / strep_iCdG692_riptide.model.slim_optimize()) * 3200.0, 2)
print('Doubling time: ' + str(doubling) + ' minutes')

Doubling time: 20.22 minutes


In [16]:
gf_iCdG692_riptide = riptide.contextualize(model=iCdG692, transcriptome=gnotobiotic)


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 338 from 1308 (74.16% change)
Metabolites pruned to 328 from 1235 (73.44% change)
Flux through the objective DECREASED to ~157.92 from ~194.8 (18.93% change)
Context-specific metabolism does not correlate with transcriptome (r=0.086, n.s.)

RIPTiDe completed in 31 seconds



In [18]:
doubling = round((1.0 / gf_iCdG692_riptide.model.slim_optimize()) * 3200.0, 2)
print('Doubling time: ' + str(doubling) + ' minutes')

Doubling time: 20.26 minutes


In [21]:
# Write results to files
riptide.save_output(riptide_obj=clinda_iCdG692_riptide, path='/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/contextualized_iCdG692/riptide_clinda')
riptide.save_output(riptide_obj=strep_iCdG692_riptide, path='/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/contextualized_iCdG692/riptide_strep')
riptide.save_output(riptide_obj=gf_iCdG692_riptide, path='/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/contextualized_iCdG692/riptide_gnoto')

### Context-specific Gene Essentiality

In [22]:
import copy
from cobra.flux_analysis.variability import find_essential_genes

def essential_genes(genre, media=None, optimum_fraction=0.01):
    model = copy.deepcopy(genre)
    
    # Change media conditions if necessary
    if media is not None:
        exchanges = ['EX_' + x for x in media]
        for rxn in model.reactions:
            if len(list(rxn.products)) == 0:
                if rxn.id in exchanges:
                    model.reactions.get_by_id(rxn.id).lower_bound = max([-1000.0,model.reactions.get_by_id(rxn.id).lower_bound])
                else:
                    model.reactions.get_by_id(rxn.id).lower_bound = 0.0
    
    # Find essential genes
    essential_genes = find_essential_genes(model, threshold=optimum_fraction)
    
    return essential_genes
                

In [23]:
# Unconstrained base model
base_essential = essential_genes(iCdG692)
base_essential_genes = set([x.id for x in base_essential])
print('Uncontextualized essential genes: ' + str(len(base_essential_genes)))

Uncontextualized essential genes: 58


In [24]:
clinda_essential_genes = essential_genes(clinda_iCdG692_riptide.model)
clinda_essential_genes = set([x.id for x in clinda_essential_genes])
print('Clinda essential genes: ' + str(len(clinda_essential_genes)))

Clinda essential genes: 87


In [25]:
strep_essential_genes = essential_genes(strep_iCdG692_riptide.model)
strep_essential_genes = set([x.id for x in strep_essential_genes])
print('Strep essential genes: ' + str(len(strep_essential_genes)))

Strep essential genes: 86


In [26]:
shared_essential_genes = strep_essential_genes.intersection(clinda_essential_genes)
strep_only_essential_genes = strep_essential_genes.difference(clinda_essential_genes)
clinda_only_essential_genes = clinda_essential_genes.difference(strep_essential_genes)

print('Shared: ' + str(len(shared_essential_genes)) + '\n')
print('Strep only: ' + str(len(strep_only_essential_genes)))
print('Clinda only: ' + str(len(clinda_only_essential_genes)))

Shared: 83

Strep only: 3
Clinda only: 4


In [27]:
print('\nstrep only:')
for gene in strep_only_essential_genes:
    print(strep_iCdG692_riptide.model.genes.get_by_id(gene).name)
# CD630_06820 = sodium:solute symporter
    
print('\nclinda only:')
for gene in clinda_only_essential_genes:
    print(clinda_iCdG692_riptide.model.genes.get_by_id(gene).name)


strep only:
PTS system, N-acetylglucosamine-specific IIC component (EC 2.7.1.69)
Glutamate synthase [NADPH] large chain (EC 1.4.1.13)
N-acetylgalactosamine-6-phosphate deacetylase | nagA

clinda only:
Glycine cleavage system H protein
pyrimidine-nucleoside phosphorylase (EC 2.4.2.2)
Deoxyribose-phosphate aldolase (EC 4.1.2.4)
phosphoribosylglycinamide formyltransferase (EC 2.1.2.2)


### Context-specific Substrate Importance

In [28]:
import pandas
from cobra.medium import minimal_medium

def find_essential_metabolites(model, fraction=0.01):

    max_growth = model.slim_optimize() * fraction
    ser = minimal_medium(model, max_growth)

    rxns = list(ser.index)
    fluxes = list(ser.values)
    cpds = []
    for x in rxns: cpds.append(model.reactions.get_by_id(x).reactants[0].name)        
    
    media = {'Reaction': rxns, 'Substrate': cpds, 'Units': fluxes}    
    media = pandas.DataFrame(media)
    print('Minimal media components: ' + str(len(media.index)))
    
    return(media)


In [29]:
clinda_min_media = find_essential_metabolites(clinda_iCdG692_riptide.model)

Minimal media components: 46


In [30]:
strep_min_media = find_essential_metabolites(strep_iCdG692_riptide.model)

Minimal media components: 46


In [31]:
strep_metabolites = set(strep_min_media['Substrate'])
clinda_metabolites = set(clinda_min_media['Substrate'])

shared_metabolites = strep_metabolites.intersection(clinda_metabolites)
strep_only_metabolites = strep_metabolites.difference(clinda_metabolites)
clinda_only_metabolites = clinda_metabolites.difference(strep_metabolites)

print('Shared: ' + str(len(shared_metabolites)) + '\n')
print('Strep only: ' + str(len(strep_only_metabolites)))
print('Clinda only: ' + str(len(clinda_only_metabolites)))

Shared: 46

Strep only: 0
Clinda only: 0


###  Context-specific Topological Differences

In [32]:
# Shared topology - reactions
clinda_rxns = set([x.id for x in clinda_iCdG692_riptide.model.reactions])
strep_rxns = set([x.id for x in strep_iCdG692_riptide.model.reactions])

shared_rxns = strep_rxns.intersection(clinda_rxns)
strep_only_rxns = strep_rxns.difference(clinda_rxns)
clinda_only_rxns = clinda_rxns.difference(strep_rxns)

print('Shared: ' + str(len(shared_rxns)) + '\n')
print('Strep only: ' + str(len(strep_only_rxns)))
print('Clinda only: ' + str(len(clinda_only_rxns)))

Shared: 304

Strep only: 9
Clinda only: 12


In [33]:
print('\nstrep only:')
print(strep_only_rxns)

print('\nclinda only:')
print(clinda_only_rxns)


strep only:
{'rxn00293_c', 'EX_cpd00165_e', 'rxn00171_c', 'rxn05703_c', 'rxn01859_c', 'rxn00109_c', 'rxn00692_c', 'rxn05317_c', 'EX_cpd00438_e'}

clinda only:
{'rxn01986_c', 'rxn00213_c', 'rxn05569_c', 'rxn01366_c', 'rxn01138_c', 'rxn05562_c', 'EX_cpd00276_e', 'rxn01368_c', 'rxn00784_c', 'rxn00193_c', 'EX_cpd00089_e', 'rxn00908_c'}


In [34]:
# Shared topology - metabolites
clinda_cpds = set([x.id for x in clinda_iCdG692_riptide.model.metabolites])
strep_cpds = set([x.id for x in strep_iCdG692_riptide.model.metabolites])

shared_cpds = strep_cpds.intersection(clinda_cpds)
strep_only_cpds = strep_cpds.difference(clinda_cpds)
clinda_only_cpds = clinda_cpds.difference(strep_cpds)

print('Shared: ' + str(len(shared_cpds)) + '\n')
print('Strep only: ' + str(len(strep_only_cpds)))
print('Clinda only: ' + str(len(clinda_only_cpds)))

Shared: 313

Strep only: 4
Clinda only: 5


In [35]:
print('Strep only:')
for x in strep_only_cpds:
    print(x + '\t' + strep_iCdG692_riptide.model.metabolites.get_by_id(x).name)

print('\nClinda only:')
for y in clinda_only_cpds:
    print(y + '\t' + clinda_iCdG692_riptide.model.metabolites.get_by_id(y).name)

Strep only:
cpd00438_c	Deoxyadenosine
cpd00438_e	Deoxyadenosine
cpd00165_e	Hydroxylamine
cpd00165_c	Hydroxylamine

Clinda only:
cpd00089_c	Glucose-1-phosphate
cpd00089_e	Glucose-1-phosphate
cpd00510_c	deoxyribose-5-phosphate
cpd00249_c	Uridine
cpd00276_e	D-Glucosamine


### Reaction essentiality

In [36]:
import copy
from cobra.flux_analysis.variability import find_essential_reactions

def essential_reactions(genre, media=None, optimum_fraction=0.01):
    model = copy.deepcopy(genre)
    
    # Change media conditions if necessary
    if media is not None:
        exchanges = ['EX_' + x for x in media]
        for rxn in model.reactions:
            if len(list(rxn.products)) == 0:
                if rxn.id in exchanges:
                    model.reactions.get_by_id(rxn.id).lower_bound = max([-1000.0,model.reactions.get_by_id(rxn.id).lower_bound])
                else:
                    model.reactions.get_by_id(rxn.id).lower_bound = 0.0
    
    # Find essential genes
    essential_reactions = find_essential_reactions(model, threshold=optimum_fraction)
    
    return essential_reactions
                

In [37]:
clinda_essential_reactions = essential_reactions(clinda_iCdG692_riptide.model)
clinda_essential_reactions = set([x.id for x in clinda_essential_reactions])
print('Clinda essential reactions: ' + str(len(clinda_essential_reactions)))

Clinda essential reactions: 224


In [38]:
strep_essential_reactions = essential_reactions(strep_iCdG692_riptide.model)
strep_essential_reactions = set([x.id for x in strep_essential_reactions])
print('Strep essential reactions: ' + str(len(strep_essential_reactions)))

Strep essential reactions: 230


In [39]:
strep_only_essential_reactions = strep_essential_reactions.difference(clinda_essential_reactions)
clinda_only_essential_reactions = clinda_essential_reactions.difference(strep_essential_reactions)
print('Strep only: ' + str(len(strep_only_essential_reactions)))
print('Clinda only: ' + str(len(clinda_only_essential_reactions)))

clinda_strep_reactions = clinda_essential_reactions.intersection(strep_essential_reactions)
print('Clinda + Strep: ' + str(len(clinda_strep_reactions)))

Strep only: 15
Clinda only: 9
Clinda + Strep: 215


In [40]:
print('Strep only:')
for x in strep_only_essential_reactions:
    print(x + '\t' + strep_iCdG692_riptide.model.reactions.get_by_id(x).name)
    
print('\nClinda only:')
for y in clinda_only_essential_reactions:
    print(y + '\t' + clinda_iCdG692_riptide.model.reactions.get_by_id(y).name)

Strep only:
rxn00293_c	UTP:N-acetyl-alpha-D-glucosamine-1-phosphate uridylyltransferase
rxn00171_c	acetaldehyde:NAD+ oxidoreductase (CoA-acetylating)
EX_cpd00122_e	N-Acetyl-D-glucosamine exchange
rxn05672_c	D-glutamate transport in via proton symport
rxn01484_c	N-Acetyl-D-glucosamine-6-phosphate amidohydrolase
rxn30278_c	Dipeptide transport via ABC system
rxn05485_c	N-Acetyl-D-glucosamine transport via PEP:Pyr PTS
rxn01859_c	Deoxyadenosine:orthophosphate ribosyltransferase
EX_cpd00186_e	D-Glutamate exchange
rxn30246_c	Aminopeptidase
rxn00692_c	5,10-Methylenetetrahydrofolate:glycine hydroxymethyltransferase
rxn00085_c	L-Glutamate:NADP+ oxidoreductase (transaminating)
rxn05317_c	Transport of deoxyadenosine, extracellular
EX_cpd00438_e	Deoxyadenosine exchange
EX_cpd29691_e	gly-L-gln exchange

Clinda only:
rxn01986_c	2-deoxy-D-ribose 1-phosphate 1,5-phosphomutase
rxn00213_c	UTP:alpha-D-glucose-1-phosphate uridylyltransferase
rxn01366_c	Uridine:phosphate alpha-D-ribosyltransferase
rxn01138_

In [41]:
# Inferring media condition

def find_growth_substrates(riptide):
    substrates = []
    exchanges = list(set([x.id for x in riptide.model.reactions if 'EX_' in x.id]))
    for rxn in exchanges:
        if numpy.median(riptide.flux_samples[rxn]) < 0.0:
            substrate_id = riptide.model.reactions.get_by_id(rxn).reactants[0].id
            substrate_name = riptide.model.reactions.get_by_id(rxn).reactants[0].name
            substrates.append([substrate_id, substrate_name])
    
    print(str(len(substrates)) + ' growth substrates found')
    substrates = pandas.DataFrame.from_records(substrates)
    substrates.columns = ['id','name']
    
    return substrates

def find_byproducts(riptide):
    byproducts = []
    exchanges = list(set([x.id for x in riptide.model.reactions if 'EX_' in x.id]))
    for rxn in exchanges:
        if numpy.median(riptide.flux_samples[rxn]) > 0.0:
            byproduct_id = riptide.model.reactions.get_by_id(rxn).reactants[0].id
            byproduct_name = riptide.model.reactions.get_by_id(rxn).reactants[0].name
            byproducts.append([byproduct_id, byproduct_name])
    
    print(str(len(byproducts)) + ' secreted byproducts found')
    byproducts = pandas.DataFrame.from_records(byproducts)
    byproducts.columns = ['id','name']
    
    return byproducts


def find_element_sources(riptide):
    
    # Isolate exchange reactions
    exchanges = []
    for rxn in riptide.model.reactions:
        if len(rxn.reactants) == 0 or len(rxn.products) == 0:
            exchanges.append(rxn.id)
    
    sources = {}
    c_source = ['cpd_id', 0.0]
    n_source = ['cpd_id', 0.0]
    
    # Parse exchange flux samples for imported metabolites
    for rxn in exchanges:
        flux = abs(numpy.median(riptide.flux_samples[rxn]))
        if flux > 1e-6:
            metabolite = riptide.model.reactions.get_by_id(rxn).reactants[0]
            sources[metabolite.id] = {}
            
            # Multiply elemental components by median flux absolute value
            for element in metabolite.elements.keys():
                element_supply = round(float(metabolite.elements[element]) * flux, 3)
                sources[metabolite.id][element] = element_supply
                
                # Identify largest sources of carbon and nitrogen
                if element == 'C' and element_supply > c_source[1]:
                    c_source = [metabolite.id, element_supply]
                elif element == 'N' and element_supply > n_source[1]:
                    n_source = [metabolite.id, element_supply]
                    
    print('Primary carbon source: ' + riptide.model.metabolites.get_by_id(c_source[0]).name + ' (' + str(c_source[1]) + ')')
    print('Primary nitrogen source: ' + riptide.model.metabolites.get_by_id(n_source[0]).name + ' (' + str(n_source[1]) + ')')

    return sources

In [42]:
clinda_substrates = find_growth_substrates(clinda_iCdG692_riptide)
clinda_sources = find_element_sources(clinda_iCdG692_riptide)
clinda_byproducts = find_byproducts(clinda_iCdG692_riptide)

49 growth substrates found
Primary carbon source: N-Acetyl-D-glucosamine (4939.564)
Primary nitrogen source: D-Aspartate (981.848)
15 secreted byproducts found


In [43]:
strep_substrates = find_growth_substrates(strep_iCdG692_riptide)
strep_sources = find_element_sources(strep_iCdG692_riptide)
strep_byproducts = find_byproducts(strep_iCdG692_riptide)

49 growth substrates found
Primary carbon source: L-Isoleucine (4615.715)
Primary nitrogen source: D-Aspartate (981.702)
14 secreted byproducts found


In [44]:
import numpy
import cobra
import cobra.flux_analysis

def _getKey(item):
    return item[1]

# Scale each active exchange back and examine its influence on objective flux
def find_primary_sources(model, flux_samples=False, fraction=0.01, cutoff=0.8, pfba_fraction=0.8):
    # Requires a model 
    # Optional: flux_samples = flux samples pandas dataframe associated with model
    #           fraction = percent of median flux to limit exchange reactions by
    #           cutoff = quantile of top C and N sources to report
    #           pfba_fraction = fraction of optimal objective value for pfba solution

    sources = {}
    c_sources = []
    c_influence = []
    n_sources = []
    n_influence = []
    p_source = ['p_source', -1.0]
    s_source = ['s_source', -1.0]
    objVal = model.slim_optimize()
    pfba_solution = cobra.flux_analysis.pfba(model, fraction_of_optimum=pfba_fraction)
    
    # Parse exchange flux samples for imported metabolites
    exchanges = [rxn.id for rxn in model.boundary]
    for rxn in exchanges:
        if isinstance(flux_samples, pandas.DataFrame):
            flux = numpy.median(flux_samples[rxn])
        else:
            flux = pfba_solution.fluxes[rxn]
        if flux >= -1e-6: continue # Skip exported byproducts or unused reactions
        bound = flux * fraction
        
        # Test for disproportionate effect on objective
        old_bounds = model.reactions.get_by_id(rxn).bounds
        model.reactions.get_by_id(rxn).bounds = (bound, bound)        
        new_objVal = model.slim_optimize()
        model.reactions.get_by_id(rxn).bounds = old_bounds # Reset bounds
        if str(new_objVal) == 'nan': new_objVal = objVal * fraction # Correct for nan
        
        # Calculate the degree of change to objective value
        if new_objVal != objVal:
            flux_ratio = objVal / new_objVal
            adjustment = abs(flux) * flux_ratio
        else:
            adjustment = 1.
        
        # Normalize elemental component contributions
        metabolite = model.reactions.get_by_id(rxn).reactants[0]
        sources[metabolite.id] = {}
        for element in metabolite.elements.keys():
            element_supply = float(metabolite.elements[element]) * adjustment
            if element_supply > 0.: element_supply = numpy.log(element_supply)
            sources[metabolite.id][element] = element_supply
                
            # Identify largest sources of main elements
            if element == 'C' and element_supply > 0.0:
                c_sources.append([metabolite.id, element_supply])
                c_influence.append(element_supply)
            elif element == 'N' and element_supply > 0.0:
                n_sources.append([metabolite.id, element_supply])
                n_influence.append(element_supply)
            elif element == 'P' and element_supply > 0.0:
                p_source = [metabolite.id, element_supply]
            elif element == 'S' and element_supply > 0.0:
                s_source = [metabolite.id, element_supply]
    
    # Rank by largest contributions
    c_sources = sorted(c_sources, reverse=True, key=_getKey)
    n_sources = sorted(n_sources, reverse=True, key=_getKey)
    
    print('Top carbon sources:')
    current = max(c_influence)
    x = 0
    while current >= numpy.quantile(c_influence, cutoff):
        print(model.metabolites.get_by_id(c_sources[x][0]).name + ' (' + str(round(c_sources[x][1],3)) + ')')
        current = c_sources[x][1]
        x += 1
        
    print('\nTop nitrogen sources:')
    current = max(n_influence)
    x = 0
    while current >= numpy.quantile(n_influence, cutoff):
        print(model.metabolites.get_by_id(n_sources[x][0]).name + ' (' + str(round(n_sources[x][1],3)) + ')')
        current = n_sources[x][1]
        x += 1
        
    print('\nPrimary phosphorous source:')
    print(model.metabolites.get_by_id(p_source[0]).name + ' (' + str(round(p_source[1],3)) + ')')
    
    print('\nPrimary sulfur source:')
    print(model.metabolites.get_by_id(s_source[0]).name + ' (' + str(round(s_source[1],3)) + ')')
    
    return sources


In [45]:
clinda_sources = find_primary_sources(clinda_iCdG692_riptide.model)

Top carbon sources:
Neu5Ac (11.351)
Octadecanoic acid (11.257)
L-Lysine (10.74)
Guanosine (10.511)
meso-2,6-Diaminopimelate (10.419)
Cytidine (9.983)
L-Isoleucine (9.855)
L-Leucine (9.765)
Gly-Phe (9.64)

Top nitrogen sources:
Guanosine (9.818)
L-Lysine (9.642)
meso-2,6-Diaminopimelate (9.166)
Neu5Ac (8.953)
Cytidine (8.885)
gly-L-asn (8.726)
Adenosine (8.425)
L-Arginine (8.314)

Primary phosphorous source:
D-fructose-6-phosphate (6.308)

Primary sulfur source:
gly-L-met (6.758)


In [46]:
strep_sources = find_primary_sources(strep_iCdG692_riptide.model)

Top carbon sources:
Neu5Ac (11.573)
Octadecanoic acid (11.25)
D-Glutamate (10.768)
L-Lysine (10.733)
meso-2,6-Diaminopimelate (10.411)
Adenosine (10.27)
gly-L-gln (9.98)
Guanosine (9.94)
L-Isoleucine (9.848)

Top nitrogen sources:
L-Lysine (9.635)
Adenosine (9.577)
Guanosine (9.247)
Neu5Ac (9.175)
meso-2,6-Diaminopimelate (9.159)
D-Glutamate (9.159)
gly-L-gln (9.132)
gly-L-asn (8.719)

Primary phosphorous source:
D-fructose-6-phosphate (6.128)

Primary sulfur source:
gly-L-met (6.751)


In [47]:
gf_sources = find_primary_sources(gf_iCdG692_riptide.model)

Top carbon sources:
L-Lysine (10.731)
meso-2,6-Diaminopimelate (10.409)
L-Isoleucine (9.846)
Cytidine (9.786)
L-Leucine (9.755)
Gly-Phe (9.63)
gly-asn-L (9.41)
Deoxycytidine (9.274)
L-Valine (9.246)

Top nitrogen sources:
L-Lysine (9.632)
meso-2,6-Diaminopimelate (9.156)
gly-asn-L (8.716)
Cytidine (8.687)
Adenosine (8.416)
L-Arginine (8.305)
Deoxycytidine (8.176)
L-Isoleucine (8.054)

Primary phosphorous source:
Phosphate (6.354)

Primary sulfur source:
Gly-Met (6.749)


In [48]:
base_sources = find_primary_sources(iCdG692)

Top carbon sources:
L-Isoleucine (10.055)
L-Valine (9.456)
2-keto-3-deoxygluconate (8.602)
L-Tryptophan (7.917)
Octadecanoic acid (7.755)
Neu5Ac (7.646)
D-glucose-6-phosphate (6.751)
D-fructose-6-phosphate (6.62)
L-ala-gly (6.606)

Top nitrogen sources:
L-Isoleucine (8.264)
L-Valine (7.846)
L-Tryptophan (6.212)
L-ala-gly (5.69)
Deoxyadenosine (5.486)
Neu5Ac (5.248)
L-Lysine (5.014)

Primary phosphorous source:
D-glucose-6-phosphate (4.96)

Primary sulfur source:
gly-L-met (2.13)
